In [ ]:
%load_ext pystackql.magic

In [ ]:
%stackql registry pull github

In [ ]:
org = "stackql"
owner = "stackql"
repo = "stackql"

## Top Stargazers per Repo

In [ ]:
%%stackql
select name, stargazers FROM
(select name, stargazers_count as stargazers
from github.repos.repos 
where org = '$org' 
and visibility = 'public'
order by stargazers_count desc) t
limit 10

In [ ]:
_.plot(kind='bar', title='Stargazers', x='name', y='stargazers');

## Daily Traffic Stats for `stackql`

In [ ]:
%%stackql
select 
JSON_EXTRACT(json_each.value, '$$.count') as count,
JSON_EXTRACT(json_each.value, '$$.uniques') as uniques,
DATE(JSON_EXTRACT(json_each.value, '$$.timestamp')) as date
from github.repos.view_traffic, json_each(views) 
WHERE owner = '$owner' AND repo = '$repo'

In [ ]:
import matplotlib.pyplot as plt

df = stackql_df
fig, ax = plt.subplots(figsize=(10,6))
ax.bar(df['date'], df['count'], color='b', alpha=0.6, label='Count')
ax.plot(df['date'], df['uniques'], color='r', marker='o', label='Uniques')

ax.set_xlabel('Date')
ax.set_ylabel('Value')
ax.set_xticks(df['date'])
ax.set_xticklabels(df['date'], rotation=45)
ax.legend()

plt.title('Count and Uniques Over Time')
fig.tight_layout()
plt.show()

## Repo Contributors who Arent Stargazers

In [ ]:
%%stackql
SELECT c.login as contrtibutor FROM 
github.repos.contributors c
LEFT OUTER JOIN github.activity.repo_stargazers s
ON s.login = c.login
WHERE owner = '$owner' AND repo = '$repo'
AND s.login IS NULL